In [1]:
import pandas as pd
import numpy as np
import os
pd.options.display.max_columns = None

In [2]:
events = pd.read_csv('../data/cleaned-data/events.csv', low_memory=False)

In [3]:
#season_list = events['season_name'].unique()
#season_list

In [4]:
#for season in season_list:
#        df = events[events['season_name'] == season] #subset dataframe into separate seasons

In [24]:
def clean_events_data(df):
        
    #Move 'match_id' & 'season_id columns to front of DF'
    col_name2='match_id'
    second_col = df.pop(col_name2)
    df.insert(0, col_name2, second_col)
    col_name='season_name'
    first_col = df.pop(col_name)
    df.insert(0, col_name, first_col)

    #First column drops
    df = df.drop(columns=['timestamp', 'possession', 'possession_team'])

    #Subset columns that need splitting 
    type_df = df['type'].apply(pd.Series)
    play_pattern_df = df['play_pattern'].apply(pd.Series)
    team_df = df['team'].apply(pd.Series)
    player_df = df['player'].apply(pd.Series)
    position_df = df['position'].apply(pd.Series)

    #Fill null values where required
    player_df[0] = player_df[0].fillna(str({'id': 0, 'name': 0}))
    position_df[0] = position_df[0].fillna(str({'id': 0, 'name': 0}))

    #Split columns and drop redundant columns
    type_df['type_id'] = type_df[0].apply(lambda x: eval(x)['id'])
    type_df['type_name'] = type_df[0].apply(lambda x: eval(x)['name'])
    type_df = type_df.drop([0], axis=1)
    play_pattern_df['play_pattern_id'] = play_pattern_df[0].apply(lambda x: eval(x)['id'])
    play_pattern_df['play_pattern_name'] = play_pattern_df[0].apply(lambda x: eval(x)['name'])
    play_pattern_df = play_pattern_df.drop([0], axis=1)
    team_df['team_id'] = team_df[0].apply(lambda x: eval(x)['id'])
    team_df['team_name'] = team_df[0].apply(lambda x: eval(x)['name'])
    team_df = team_df.drop([0], axis=1)
    player_df['player_id'] = player_df[0].apply(lambda x: eval(x)['id'])
    player_df['player_name'] = player_df[0].apply(lambda x: eval(x)['name'])
    player_df = player_df.drop([0], axis=1)
    position_df['position_id'] = position_df[0].apply(lambda x: eval(x)['id'])
    position_df['position_name'] = position_df[0].apply(lambda x: eval(x)['name'])
    position_df = position_df.drop([0], axis=1)

    #Concat dataframes back togther and drop redundant columns
    df = pd.concat([df, type_df, play_pattern_df, team_df, player_df, position_df], axis=1).drop(
        columns = ['type', 'play_pattern', 'team', 'player', 'position'])

    #Special cleaning required on 'tactics' subset
    #tactics_df['tactics'] = tactics_df['tactics'].apply(pd.Series) DELETED
    tactics_df = df[['match_id', 'type_id', 'type_name', 'team_name', 'tactics']]
    tactics_df['tactics'] = tactics_df['tactics'].fillna(str({'formation': 0, 'lineup': 0}))

    import ast
    tactics_df['tactics'] = [ast.literal_eval(i) for i in tactics_df['tactics']]
    tactics_df['formation'] = [tactics_df['tactics'].iloc[i]['formation'] for i in range(len(tactics_df['tactics']))]
    tactics_df['lineup'] = [tactics_df['tactics'].iloc[i]['lineup'] for i in range(len(tactics_df['tactics']))]
    tactics_player_split = tactics_df['lineup'].apply(pd.Series).add_prefix('player_')
    tactics_df = pd.concat([tactics_df, tactics_player_split], axis=1).drop(columns='tactics')
    #tactics_df = tactics.drop(columns='match_id') DELETED

    #Concat dataframes back togther and drop redundant columns
    df = pd.concat([df, tactics_df], axis=1).drop(columns = ['tactics'])

    #Save cleaned dataframe to CSV file
    df = df.to_csv('../data/cleaned-data/events_cleaned.csv', index=False)

In [22]:
clean_events_data(events)

C:\Users\paul_\anaconda3\envs\matrix\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\paul_\anaconda3\envs\matrix\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\paul_\anaconda3\envs\matrix\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta